# Harry Potter API

In [44]:
import requests
import json
from pymongo import MongoClient
from mongoConnection import *
from flask import Flask, request, jsonify
from bson.json_util import dumps

## Conecto con Mongo

In [2]:
client = MongoClient("mongodb://localhost/harrypotter")
db = client.get_database()
client.harrypotter
quots = client.harrypotter.quotes
chars = client.harrypotter.cast
movs = client.harrypotter.movies

## Bienvenid@ a mi API!

In [3]:
res = requests.get("http://127.0.0.1:5000/welcome")
res.text

'Welcome to the magical world of Harry Potter'

### Listo las colecciones de mi base de datos para ver que funciona

In [4]:
list(db.list_collections())

[{'name': 'movies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('0455be61-1c18-40fc-9f1b-e0f664a5bbd3')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}},
 {'name': 'cast',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('550c4a80-4343-41c9-9117-1b2248447ac9')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}},
 {'name': 'quotes',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('a2ce707d-f2fb-447c-9d38-15801a892dbd')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}},
 {'name': 'characters',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('e3ccc85d-ef92-4ee8-a7f4-3fc8abf13dcd')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

## Colección Quotes

### Busco todas las frases dichas en las peliculas

In [5]:
res = requests.get("http://127.0.0.1:5000/phrase")
res.text[:1000]

'[\n  {\n    "Characters": "Harry Potter", \n    "Movie": "Harry Potter and the Chamber of Secrets", \n    "Phrase": "I\\u2019ll be in my bedroom, making no noise and pretending I\\u2019m not there.", \n    "Year": "2002"\n  }, \n  {\n    "Characters": "Harry Potter", \n    "Movie": "Harry Potter and the Prisoner of Azkaban", \n    "Phrase": "I solemnly swear I am up to no good.", \n    "Year": "2004"\n  }, \n  {\n    "Characters": "Harry Potter", \n    "Movie": "Harry Potter and the Prisoner of Azkaban", \n    "Phrase": "Mischief Managed!", \n    "Year": "2004"\n  }, \n  {\n    "Characters": "Harry Potter", \n    "Movie": "Harry Potter and the Half-Blood Prince", \n    "Phrase": "There is no need to call me \\u2018sir,\\u2019 Professor.", \n    "Year": "2009"\n  }, \n  {\n    "Characters": "Hermione Gringer", \n    "Movie": "Harry Potter and the Goblet of Fire", \n    "Phrase": "Twitchy little ferret, aren\\u2019t you, Malfoy?", \n    "Year": "2005"\n  }, \n  {\n    "Characters": "Her

### Busco una frase de la pelicula

#### Primero hago una query a mongo para ver las frases.

In [6]:
info = list(db.quotes.find({}, {"Phrase":1, "_id":0, "Characters":1}))
info[:10]

[{'Characters': 'Harry Potter',
  'Phrase': 'I’ll be in my bedroom, making no noise and pretending I’m not there.'},
 {'Characters': 'Harry Potter',
  'Phrase': 'I solemnly swear I am up to no good.'},
 {'Characters': 'Harry Potter', 'Phrase': 'Mischief Managed!'},
 {'Characters': 'Harry Potter',
  'Phrase': 'There is no need to call me ‘sir,’ Professor.'},
 {'Characters': 'Hermione Gringer',
  'Phrase': 'Twitchy little ferret, aren’t you, Malfoy?'},
 {'Characters': 'Hermione Gringer',
  'Phrase': 'Fear of a name only increases fear of the thing itself.'},
 {'Characters': 'Hermione Gringer',
  'Phrase': "Just because you have the emotional range of a teaspoon doesn't mean we all have."},
 {'Characters': 'Ron Weasley',
  'Phrase': "You're a little scary sometimes, you know that? Brilliant ... but scary."},
 {'Characters': 'Ron Weasley', 'Phrase': 'When in doubt, go to the library.'},
 {'Characters': 'Ron Weasley',
  'Phrase': 'Don’t let the muggles get you down.'}]

### Pruebo con la API a buscar una frase.

In [7]:
res = requests.get("http://127.0.0.1:5000/phrase/Don’t let the muggles get you down.")
res.text

'[\n  {\n    "Characters": "Ron Weasley", \n    "Movie": "Harry Potter and the Prisoner of Azkaban", \n    "Phrase": "Don\\u2019t let the muggles get you down.", \n    "Year": "2004"\n  }\n]\n'

### Creo un nueva frase y la inserto

In [22]:
new_message = {
    "Characters": "Albus Dumbledore",
    "Phrase": "You need your friends, Harry",
    "Movie": "Harry Potter and the Half-Blood Prince",
    "Year": "2009"
    }

In [23]:
res = requests.get("http://127.0.0.1:5000/add/new_quote/<Characters>/<Phrase>/<Movie>/<Year>")
res

<Response [200]>

In [28]:
insert_quote("Albus Dumbledore","You need your friends, Harry","Harry Potter and the Half-Blood Prince","2009")

In [27]:
insert_quote("Ron Weasley","I always said he was mental. Brilliant and everything, but cracked.","Harry Potter and the Deathly Hallows - Part 1", "2010")

### Elimino un frase

In [39]:
res = requests.get("http://127.0.0.1:5000/delete_quote/<Characters>/<Phrase>/<Movie>/<Year>")
res

<Response [200]>

In [29]:
delete_quote("Albus Dumbledore","You need your friends, Harry","Harry Potter and the Half-Blood Prince","2009")

{'n': 1, 'ok': 1.0}

## Colección Cast

### Busco todos los personajes

In [30]:
res2 = requests.get("http://127.0.0.1:5000/characters")
res2.text[:1000]

'[\n  {\n    "Actor name": "Daniel Radcliffe", \n    "Characters": "Harry Potter", \n    "Role": "Student"\n  }, \n  {\n    "Actor name": "Emma Watson", \n    "Characters": "Hermione Gringer", \n    "Role": "Student"\n  }, \n  {\n    "Actor name": "Rupert Grint", \n    "Characters": "Ron Weasley", \n    "Role": "Student"\n  }, \n  {\n    "Actor name": "Richard Harris, Michael Gambon", \n    "Characters": "Albus Dumbledore", \n    "Role": "Director of Hogwarts"\n  }, \n  {\n    "Actor name": "Maggie Smith", \n    "Characters": "Minerva McGonagall", \n    "Role": "Teacher"\n  }, \n  {\n    "Actor name": "Alan Rickman", \n    "Characters": "Severus Snape", \n    "Role": "Teacher"\n  }, \n  {\n    "Actor name": "Robbie Coltrane", \n    "Characters": "Rubeus Hagrid", \n    "Role": "Key keeper"\n  }, \n  {\n    "Actor name": "Tom Felton", \n    "Characters": "Draco Malfoy", \n    "Role": "Student"\n  }, \n  {\n    "Actor name": "Toby Jones", \n    "Characters": "Dobby", \n    "Role": "Elf"\n

### Pruebo con la API a buscar un personaje

In [31]:
res3 = requests.get("http://127.0.0.1:5000/characters/Harry Potter")
res3.text

'[\n  {\n    "Actor name": "Daniel Radcliffe", \n    "Characters": "Harry Potter", \n    "Role": "Student"\n  }\n]\n'

### Inserto un nuevo personaje

In [37]:
new_char = {
    "Characters":"Rolanda Hooch",
    "Role":"Referee for Hogwarts Quidditch matches",
    "Actor name":"Zoë Wanamaker"   
}

In [38]:
res = requests.get("http://127.0.0.1:5000/add/new_char/<Characters>/<Role>/<Actorname>")
res

<Response [200]>

In [35]:
insert_char("Luna Lovegood","Student","Evanna Lynch")

In [36]:
insert_char("Rolanda Hooch","Referee for Hogwarts Quidditch matches","Zoë Wanamaker")

### Elimino un nuevo personaje

In [40]:
res = requests.get("http://127.0.0.1:5000/delete_char/<Characters>/<Role>/<Actorname>")
res

<Response [200]>

In [41]:
delete_char("Rolanda Hooch","Referee for Hogwarts Quidditch matches","Zoë Wanamaker")

{'n': 4, 'ok': 1.0}